In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

In [ ]:
%%writefile restricted_zone_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
mkdir -p $1
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4

if [ "$2" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/altera/aocl-pro-rte/aclrte-linux64/
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/computer_vision_sdk/bitstreams/a10_vision_design_bitstreams/5-0_PL1_FP11_RMNet.aocx
fi
    
# Running the object detection code
python3 main.py -m /opt/intel/computer_vision_sdk/deployment_tools/intel_models/person-detection-retail-0013/$3/person-detection-retail-0013.xml \
                                           -i $4 \
                                           -x 222 \
                                           -y 102 \
                                           -ht 612 \
                                           -w 610 \
                                           -o $1 \
                                           -d $2 \
                                           -l /opt/intel/computer_vision_sdk/inference_engine/lib/ubuntu_16.04/intel64/libcpu_extension_avx2.so
g++ -std=c++14 ROI_writer.cpp -o ROI_writer  -lopencv_core -lopencv_videoio -lopencv_imgproc -lopencv_highgui  -fopenmp -I/opt/intel/computer_vision_sdk/opencv/include/ -L/opt/intel/computer_vision_sdk/opencv/lib/

# Rendering the output video
SKIPFRAME=1
RESOLUTION=0.5
./ROI_writer $INPUT_FILE $OUTPUT_FILE $SKIPFRAME $RESOLUTION

In [ ]:
os.environ["VIDEO"] = "resources/worker-zone-detection.mp4"

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te -F "results/core CPU FP32 $VIDEO" -N res_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/core', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_fpga = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "results/fpga/ HETERO:FPGA,CPU FP32 $VIDEO" -N obj_det_fpga
print(job_id_fpga[0]) 
#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga/', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/fpga/', 'v_progress_'+job_id_fpga[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_ncs = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs -F "results/myriad/ MYRIAD FP16 $VIDEO" -N res_ncs
print(job_id_ncs[0]) 
#Progress indicators
if job_id_ncs:
    progressIndicator('results/myriad/', 'i_progress_'+job_id_ncs[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/myriad/', 'v_progress_'+job_id_ncs[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub restricted_zone_job.sh  -l nodes=1:tank-870:e3-1268l-v5 -F "results/xeon/ CPU FP32 $VIDEO" -N res_xeon 
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/xeon/', 'v_progress_'+job_id_xeon[0]+'.txt', "Rendering", 0, 100)
    

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "results/gpu/ GPU FP32 $VIDEO" -N res_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/gpu/', 'v_progress_'+job_id_gpu[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "results/ncs2/ MYRIAD FP16 $VIDEO" -N res_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/ncs2/', 'v_progress_'+job_id_ncs2[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_vpu = !qsub restricted_zone_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "results/vpu/ HDDL FP16 $VIDEO" -N res_vpu
print(job_id_vpu[0]) 
#Progress indicators
if job_id_vpu:
    progressIndicator('results/vpu/', 'i_progress_'+job_id_vpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/vpu/', 'v_progress_'+job_id_vpu[0]+'.txt', "Rendering", 0, 100)

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub restricted_zone_job.sh -l nodes=1:up-squared -F "results/up2/ GPU FP32 $VIDEO" -N res_up2
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('results/up2/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/up2/', 'v_progress_'+job_id_up2[0]+'.txt', "Rendering", 0, 100)

In [ ]:
liveQstat()

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
          ['results/core/output_'+job_id_core[0]+'.mp4'], 'results/core/stats_'+job_id_core[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-V100-MX8 (Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))', 
          ['results/vpu/output_'+job_id_vpu[0]+'.mp4'], 'results/vpu/stats_'+job_id_vpu[0]+'.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/gpu/output_'+job_id_gpu[0]+'.mp4'], 'results/gpu/stats_'+job_id_gpu[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel NCS2', 
          ['results/ncs2/output_'+job_id_ncs2[0]+'.mp4'], 'results/ncs2/stats_'+job_id_ncs2[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel Movidius NCS', 
          ['results/myriad/output_'+job_id_ncs[0]+'.mp4'], 'results/myriad/stats_'+job_id_ncs[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)', 
          ['results/xeon/output_'+job_id_xeon[0]+'.mp4'], 'results/xeon/stats_'+job_id_xeon[0]+'.txt')

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)', 
          ['results/up2/output_'+job_id_up2[0]+'.mp4'], 'results/up2/stats_'+job_id_up2[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)', 
          ['results/fpga/output_'+job_id_fpga[0]+'.mp4'], 'results/fpga/stats_'+job_id_fpga[0]+'.txt')